In [1]:
#%pip install opensearch-py -Uqq

In [2]:
# %pip install rich -Uqq

In [25]:
from fastcore.net import *
from fastcore.test import *
from nbdev.showdoc import show_doc
from rich import inspect as rinspect
from rich import print as rprint

In [4]:
es='http://localhost:9200'

urljson(es)

{'name': 'fedora',
 'cluster_name': 'opensearch',
 'cluster_uuid': 'QX9O66TzSZiJ8gNePiB63Q',
 'version': {'distribution': 'opensearch',
  'number': '1.2.4',
  'build_type': 'tar',
  'build_hash': 'e505b10357c03ae8d26d675172402f2f2144ef0f',
  'build_date': '2022-01-14T03:38:06.881862Z',
  'build_snapshot': False,
  'lucene_version': '8.10.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [5]:
urljson(f'{es}/_cluster/health')

{'cluster_name': 'opensearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'discovered_master': True,
 'active_primary_shards': 3,
 'active_shards': 3,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 60.0}

In [47]:
from opensearchpy import *

In [48]:
host, port = 'localhost', 9200
cli = OpenSearch([{'host': host, 'port': port}])

In [49]:
rinspect(cli.cat.health)

╭─ <bound method CatClient.health of <opensearchpy.client.cat.CatClient object at 0x7f72152─╮
│ def CatClient.health(params=None, headers=None):                                          │
│                                                                                           │
│ Returns a concise representation of the cluster health.                                   │
│                                                                                           │
│ 28 attribute(s) not shown. Run inspect(inspect) for options.                              │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

In [50]:
rinspect(cli.cat.indices)

╭─ <bound method CatClient.indices of <opensearchpy.client.cat.CatClient object at 0x7f7215─╮
│ def CatClient.indices(index=None, params=None, headers=None):                             │
│                                                                                           │
│ Returns information about indices: number of primaries and replicas, document             │
│ counts, disk size, ...                                                                    │
│                                                                                           │
│ 28 attribute(s) not shown. Run inspect(inspect) for options.                              │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

In [51]:
cli.cat.health()

'1644715941 01:32:21 opensearch yellow 1 1 true 4 4 0 0 3 0 - 57.1%\n'

In [52]:
rinspect(cli.cat.count)

╭─ <bound method CatClient.count of <opensearchpy.client.cat.CatClient object at 0x7f721520─╮
│ def CatClient.count(index=None, params=None, headers=None):                               │
│                                                                                           │
│ Provides quick access to the document count of the entire cluster, or                     │
│ individual indices.                                                                       │
│                                                                                           │
│ 28 attribute(s) not shown. Run inspect(inspect) for options.                              │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

In [53]:
# Create an index with non-default settings.

index_name = 'searchml_revisited'
index_body = {
    'settings': {
        'index': {
            'query': {
                'default_field': "body"
            }
        }
    }
}
def _exists_index(index):
    return cli.cat.indices(index, h='index').strip() == index

if not _exists_index('searchml_revisited'):
    cli.indices.create(index_name, body=index_body)

In [54]:
test_eq(cli.cat.indices('searchml_revisited', h='index').strip(), 'searchml_revisited')

In [55]:
# Add our sample document to the index.
docs = [
    {
        "id": "doc_a",
        "title": "Fox and Hounds",
        "body": "The quick red fox jumped over the lazy brown dogs.",
        "price": "5.99",
        "in_stock": True,
        "category": "childrens"},
    {
        "id": "doc_b",
        "title": "Fox wins championship",
        "body": "Wearing all red, the Fox jumped out to a lead in the race over the Dog.",
        "price": "15.13",
        "in_stock": True,
        "category": "sports"},
    {
        "id": "doc_c",
        "title": "Lead Paint Removal",
        "body": "All lead must be removed from the brown and red paint.",
        "price": "150.21",
        "in_stock": False,
        "category": "instructional"},
    {
        "id": "doc_d",
        "title": "The Three Little Pigs Revisted",
        "price": "3.51",
        "in_stock": True,
        "body": "The big, bad wolf huffed and puffed and blew the house down. The end.",
        "category": "childrens"}
]

In [56]:
rinspect(cli.index)

╭─ <bound method OpenSearch.index of <OpenSearch([{'host': 'localhost', 'port': 9200}])>> ─╮
│ def OpenSearch.index(index, body, doc_type=None, id=None, params=None, headers=None):    │
│                                                                                          │
│ Creates or updates a document in an index.                                               │
│                                                                                          │
│ 28 attribute(s) not shown. Run inspect(inspect) for options.                             │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

In [57]:
index_name='searchml_revisited'
[cli.index(index_name, body=doc, id=doc['id'], refresh=True) for doc in docs]

TransportError: TransportError(429, 'cluster_block_exception', 'index [searchml_revisited] blocked by: [TOO_MANY_REQUESTS/12/disk usage exceeded flood-stage watermark, index has read-only-allow-delete block];')